# Imports required for training the model

In [1]:
import numpy as np                    # For maths, arrays, image manipulation
import matplotlib.pyplot as plt       # To visualize doodles/images
import json                           # For reading .ndjson/raw data files
import tensorflow as tf               # For deep learning (CNN)
import os                             # File system access - finding/loading files
from sklearn.model_selection import train_test_split   # For splitting data sets
from tensorflow.keras import layers, models


# Here is the sample dataset!
Example for cat

In [ ]:
# Download one category (e.g. 'cat') as example
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cat.npy

In [ ]:
cat_images = np.load('cat.npy')
print("Cat image array shape:", cat_images.shape)  # Should be (num_samples, 784)
print(cat_images[:30])

# Visualize
plt.imshow(cat_images[0].reshape(28,28), cmap='gray')
plt.title("Example QuickDraw Cat")
plt.axis('off')
plt.show()


# Data - prep
We’ll train on 10 categories for speed

In [ ]:
categories = ["cat", "dog", "tree", "car", "cloud",
              "house", "star", "airplane", "rainbow", "chair"]

# Map category names to numbers
label_map = {cat: idx for idx, cat in enumerate(categories)}
print(label_map)

Download & Load

In [ ]:
all_images = []
all_labels = []

for category in categories:
    url = f"https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{category}.npy"
    filename = f"{category}.npy"

    # Download only if not already there
    if not os.path.exists(filename):
        !wget $url -O $filename

    # Load numpy array
    images = np.load(filename)

    # Normalize pixel values [0,255] → [0,1]
    images = images.astype("float32") / 255.0

    # Limit samples for speed
    n_samples = 10000
    all_images.append(images[:n_samples])
    all_labels.append(np.full(n_samples, label_map[category]))

    print(f"{category}: {images.shape[0]} total, using {n_samples}")


Combine and reshape

In [ ]:
# Combine into one dataset
X = np.concatenate(all_images, axis=0)
y = np.concatenate(all_labels, axis=0)

print("Final dataset shape:", X.shape, y.shape)

# Reshape to (28,28,1) for CNN input
X = X.reshape(-1, 28, 28, 1)

print("Reshaped dataset:", X.shape)


Train/Test Split

In [ ]:
# Split train/test (stratify = keep class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)


Visualize Dataset

In [ ]:
# Pick 25 random samples to visualize
plt.figure(figsize=(10, 10))
for i in range(25):
    idx = np.random.randint(0, len(X))
    plt.subplot(5, 5, i+1)
    plt.imshow(X[idx].reshape(28, 28), cmap="gray")
    plt.title(categories[y[idx]])
    plt.axis("off")
plt.tight_layout()
plt.show()

# Basic CNN Model

In [ ]:
from tensorflow.keras import layers, models

# Define CNN with 3 convolutional layers
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)), #learns edges, lines, corners (low-level features).
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation="relu"), #combines them into shapes/patterns (loops, strokes).
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation="relu"), #more abstract features (e.g. “cat ear”, “wheel”, “star point”).
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(len(categories), activation="softmax")  # output layer
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

# Transfer learning + data augumentation

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras import layers, models

# X_train_rgb = tf.image.grayscale_to_rgb(tf.constant(X_train)).numpy()
# X_test_rgb = tf.image.grayscale_to_rgb(tf.constant(X_test)).numpy()

# IMG_SIZE = 75
# X_train_resized = tf.image.resize(X_train_rgb, (IMG_SIZE, IMG_SIZE)).numpy()
# X_test_resized = tf.image.resize(X_test_rgb, (IMG_SIZE, IMG_SIZE)).numpy()

# print("Original training data shape:", X_train.shape)
# print("New resized training data shape:", X_train_resized.shape)
# print("-" * 30)


# # --- 2. Build the Transfer Learning Model ---

# base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
#                          include_top=False,
#                          weights='imagenet')

# # Freeze the convolutional base to prevent its weights from changing during training.
# base_model.trainable = False

# transfer_model = models.Sequential([
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.3),
#     layers.Dense(len(categories), activation='softmax')  # Our output layer
# ])

# # Compile the model.
# transfer_model.compile(optimizer="adam",
#                        loss="sparse_categorical_crossentropy",
#                        metrics=["accuracy"])

# transfer_model.summary()

In [ ]:
# This can be used to augument the image data addded to the model.fit!


#from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Create an ImageDataGenerator for data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=False, # Not always useful for doodles, but good to know
#     fill_mode='nearest'
# )

# Train the model

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,               # enough to see improvement but not too long in class
    batch_size=128,          # process 128 doodles at a time
    validation_data=(X_test, y_test)
)

# Transfer learning

# history = transfer_model.fit(
#     datagen.flow(X_train, y_train, batch_size=128), # transfer learning  + data augumentation!
#     epochs=20,
#     validation_data=(X_test_resized, y_test)
# )

# Plot accuracy and loss over time
plt.figure(figsize=(12,5))

# Accuracy plot
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label="Train Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Model Accuracy Over Time")
plt.legend()

# Loss plot
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label="Train Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model Loss Over Time")
plt.legend()

plt.show()

Evaluate & Test Predictions

In [ ]:
# Evaluate on test set
class_names = categories
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Make predictions on a few random samples
import random

# Pick 5 random test images
indices = random.sample(range(len(X_test)), 5)

plt.figure(figsize=(12,3))
for i, idx in enumerate(indices):
    img = X_test[idx].reshape(28,28)
    label = y_test[idx]
    pred = np.argmax(model.predict(X_test[idx].reshape(1,28,28,1)))

    plt.subplot(1,5,i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Pred: {class_names[pred]}\nTrue: {class_names[label]}")
    plt.axis('off')
plt.show()


# Sample Testing live

In [ ]:
import gradio as gr

def predict_doodle(img):
    if img is None:
        return "Please draw something!"
    # If input is dict, use 'composite' image
    if isinstance(img, dict):
        img = img.get("composite", None)
    if img is None:
        return "Please draw something!"
    # Convert the composite PIL Image to grayscale numpy array (already L mode)
    img = np.array(img)
    # Resize to 28x28 and normalize
    img = tf.image.resize(img[..., np.newaxis], (28, 28)).numpy()
    img = img / 255.0
    # Predict probabilities once
    preds = model.predict(img.reshape(1, 28, 28, 1))[0]
    # Return label with highest probability
    pred_label = class_names[np.argmax(preds)]
    return pred_label


interface = gr.Interface(
    fn=predict_doodle,
    inputs=gr.Sketchpad(type="pil", image_mode="L"),
    outputs="label",
    live=True,
)
interface.launch(share=True, debug=True)

# Export the model

H5 model export for general reusablity

In [ ]:

from google.colab import files

trained_model = model

# 1. Define the filename for your model.
model_filename = "doodle_classifier.h5"

# 2. Save the model to the specified file.
trained_model.save(model_filename)

print(f"Model successfully saved to '{model_filename}'")
print("Starting download...")

# 3. Trigger the download of the file to your local machine.
files.download(model_filename)

Tf lite export for mobile apps!

In [ ]:
import tensorflow as tf

# Create a TFLiteConverter object from the Keras model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Optimizee
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 3. Convert the model.
tflite_model = converter.convert()

# 4. Save the converted model to a .tflite file.
with open('doodle_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Successfully converted model to 'doodle_model.tflite'")
print(f"File size: {len(tflite_model) / 1024:.2f} KB")

# 5. Download the file from Colab to your local machine.
from google.colab import files
files.download('doodle_model.tflite')

Tf js export for web apps

In [ ]:
!pip install tensorflowjs

import tensorflowjs as tfjs

# Define the path where you want to save the model
output_path = 'doodle_model_js'

# Convert the Keras model (use 'model' or 'transfer_model')
tfjs.converters.save_keras_model(model, output_path)

print(f"Model saved for TensorFlow.js in the '{output_path}' directory.")

# To download, first zip the directory, then download the zip file.
!zip -r {output_path}.zip {output_path}

from google.colab import files
files.download(f'{output_path}.zip')

# NOTE: It creates a directory named doodle_model_js containing a model.json file (the model architecture) and several .bin files (the weights) these bin are named like shards.smtg1.bin, etc.